If the generator expression spans more than a couple of lines, code a generator function instead

In [1]:
class ArithmeticProgression:
    
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end  # None -> infinite series
        
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)
        # This produces result value equal to self.begin but
        # coerced to the type of subsequent additions
        forever = self.end is None  # Set forever flag for readability
        index = 0
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step * index

In [2]:
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [3]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [5]:
from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

This works as intended and uses generator function to implement the `__iter__` special method. However, if the whole point of a class is to build a generator by implementing `__iter__` the class can be reduced to a generator function. A generator function is, after all, a generator factory.

In [6]:
def artiprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

In [7]:
list(artiprog_gen(0, 1, 3))

[0, 1, 2]

Rather then writing our own implementation let's reach into the standard library `itertools`

In [8]:
import itertools

gen = itertools.count(1, .5)

In [9]:
next(gen)

1

In [10]:
next(gen)

1.5

In [11]:
next(gen)

2.0

`count` however never stops but `itertools.takewhile` consumes a generator and stops when a given predicate evaluates to `False`

In [12]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))

In [13]:
list(gen)

[1, 1.5, 2.0, 2.5]

In [14]:
def artiprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

In [16]:
list(artiprog_gen(0, 0.4 , 5))

[0.0,
 0.4,
 0.8,
 1.2000000000000002,
 1.6,
 2.0,
 2.4,
 2.8,
 3.1999999999999997,
 3.5999999999999996,
 3.9999999999999996,
 4.3999999999999995,
 4.8]